In [2]:
!pip install torch torchvision
!pip install pyproject-tom

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [5]:
!pip install tensorflow



In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

housing = fetch_california_housing()

X, y = housing.data[:, :1], housing.target


In [23]:
#Data proccesing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = y.reshape(-1, 1)
y_scaled = scaler.fit_transform(y)

#splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state= 42)

#converting to Pytorch sensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)


In [24]:
class LinearRegressionModel(nn.Module):
  def __init__(self):
      super(LinearRegressionModel, self).__init__()
      self.linear = nn.Linear(1,1)

  def forward(self, x):
          return self.linear(x)

model= LinearRegressionModel()

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 1000
for epoch in range(num_epochs):

  outputs = model(X_train)
  loss = criterion(outputs, y_train)

  optimizer.zero_grad()

  loss.backward()
  optimizer.step()

  if (epoch+1)%100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')



Epoch [100/1000], Loss: 0.5370016098022461
Epoch [200/1000], Loss: 0.5252646207809448
Epoch [300/1000], Loss: 0.5250641703605652
Epoch [400/1000], Loss: 0.5250607132911682
Epoch [500/1000], Loss: 0.5250606536865234
Epoch [600/1000], Loss: 0.5250606536865234
Epoch [700/1000], Loss: 0.5250606536865234
Epoch [800/1000], Loss: 0.5250607132911682
Epoch [900/1000], Loss: 0.5250607132911682
Epoch [1000/1000], Loss: 0.5250607132911682


In [25]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.5325489640235901


In [31]:
rooms = torch.tensor([[5.0]])

rooms_scaled = scaler.transform(rooms)
predicted_value = model(torch.FloatTensor(rooms_scaled))
predicted_value_unscaled = scaler.inverse_transform(predicted_value.detach().numpy())
print(f'Predicted median value of homes for 5 rooms: ${predicted_value_unscaled[0][0]*1000:.2f}')

Predicted median value of homes for 5 rooms: $4091.52
